## <center> Precision, Recall (Sensitivity, Specificity) and Error calculation for JIP models </center>

This Notebook calculates the Precision, Recall (Sensitivity, Specificity) and Error of the predictions made by the JIP models for the test datasets.
Before executing this Notebook, be sure to have trained all 6 artefact models using the provided code in three steps:
    
1. Preprocess all datasets (train and test) using the following command:
```bash
python JIP.py --mode preprocess --device <cuda_id> --datatype train
```  
and   
```bash
python JIP.py --mode preprocess --device <cuda_id> --datatype test
```
2. Train all 6 models using the following command:
```bash
python JIP.py --mode train --device <cuda_id> --datatype train 
                 --noise_type <noise_model> --store_data
```
3. Perform the testing as follows:
```bash
python JIP.py --mode testIOOD --device <cuda_id> --datatype test
                 --noise_type <noise_model> --store_data
```


Once this is finished, everything is set up to run the Notebook.

#### Import necessary libraries

In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
import numpy as np
from itertools import zip_longest
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import classification_report

# -- Grouper from https://stackoverflow.com/questions/8991506/iterate-an-iterator-by-chunks-of-n-in-python -- #
def grouper(n, iterable, fillvalue=None):
    "grouper(3, 'ABCDEFG', 'x') --> ABC DEF Gxx"
    args = [iter(iterable)] * n
    return zip_longest(fillvalue=fillvalue, *args)

#### Set necessary directories
Specify the train_base and test_base directory. These are just the full paths to the JIP folder train_dirs and test_dirs output, for instance: `../JIP/train_dirs/output` and `../JIP/test_dirs/output`.

In [2]:
# Set the base path to JIP/train_dirs/output folder
train_base = '<path>/JIP/train_dirs/output/'
# Set the base path to JIP/test_dirs/output folder
test_base = '<path>/JIP/test_dirs/output/'

#### Load data

In [3]:
artefacts = ['blur', 'ghosting', 'motion', 'noise', 'resolution', 'spike']

data = dict()
for artefact in artefacts:
    # Load data
    dl = np.load(os.path.join(train_base, artefact, 'results', 'accuracy_detailed_test.npy'))
    ID = np.load(os.path.join(test_base, artefact, 'testID_results', 'accuracy_detailed_test.npy'))
    OOD = np.load(os.path.join(test_base, artefact, 'testOOD_results', 'accuracy_detailed_test.npy'))
    
    # Create One Hot vectors from predicted values
    for idx, a in enumerate(dl):
        b = np.zeros_like(a[1])
        b[a[1].argmax()] = 1
        a[1] = b
        dl[idx] = a
    for idx, a in enumerate(ID):
        b = np.zeros_like(a[1])
        b[a[1].argmax()] = 1
        a[1] = b
        ID[idx] = a
    for idx, a in enumerate(OOD):
        b = np.zeros_like(a[1])
        b[a[1].argmax()] = 1
        a[1] = b
        OOD[idx] = a
        
    # Save data in dictionary
    data['test_dl-' + artefact] = dl
    data['test_ID-' + artefact] = ID
    data['test_OOD-' + artefact] = OOD

#### Transform data into Confusion Matrix

In [4]:
# Transform the data into right format for calculations
y_yhats = dict()
# Loop through all data sets
for k, v in data.items():
    y_yhats[k] = dict()
    y_yhats[k]['prediction'] = list()
    y_yhats[k]['ground_truth'] = list()
    # Change the format of y_yhats --> split in prediction and GT
    for y_yhat in v:
        y_yhats[k]['prediction'].append(y_yhat[1])
        y_yhats[k]['ground_truth'].append(y_yhat[0])
    y_yhats[k]['prediction'] = np.array(y_yhats[k]['prediction'])
    y_yhats[k]['ground_truth'] = np.array(y_yhats[k]['ground_truth'])

#### Generate a Classification Report including Precision, Recall and F1 Score

In [5]:
print('NOTE: In every confusion matrix: x-axis --> predicted, y-axis --> actual.')
# Loop through the transformed data and calculate everything
for test_name, results in y_yhats.items():
    confusion = confusion_matrix(results['ground_truth'].argmax(axis=1),
                                 results['prediction'].argmax(axis=1),
                                 labels=[0, 1, 2, 3, 4])
    print('\n{}:'.format(test_name))
    print('Confusion Matrix\n')
    print(confusion)
    
    print('\nClassification Report\n')
    print(classification_report(results['prediction'], results['ground_truth'],
                                target_names=['Quality 1', 'Quality 2', 'Quality 3', 'Quality 4', 'Quality 5']))
    
    # Calculate mean values for each element in the loop
    print('\nSummarized Report\n')
    report = classification_report(results['prediction'], results['ground_truth'],
                        target_names=['Quality 1', 'Quality 2', 'Quality 3', 'Quality 4', 'Quality 5'],
                        output_dict=True)
    
    precision, recall, f1 = list(), list(), list()
    idx = 0
    for k, v in report.items():
        if v['support'] != 0 and idx < 5:
            precision.append(v['precision'])
            recall.append(v['recall'])
            f1.append(v['f1-score'])
        idx += 1
    print('Mean precision (without avg values): {}'.format(sum(precision)/len(precision)))
    print('Mean recall (without avg values): {}'.format(sum(recall)/len(recall)))
    print('Mean f1-score (without avg values): {}'.format(sum(f1)/len(f1)))
    print('--------------------------------------------------------------')
    
    FP = confusion.sum(axis=0) - np.diag(confusion)
    FN = confusion.sum(axis=1) - np.diag(confusion)
    TP = np.diag(confusion)
    TN = confusion.sum() - (FP + FN + TP)
    FP = FP.astype(float)
    FN = FN.astype(float)
    TP = TP.astype(float)
    TN = TN.astype(float)
    """
    # Sensitivity, hit rate, recall, or true positive rate
    TPR = TP/(TP+FN)
    # Specificity or true negative rate
    TNR = TN/(TN+FP) 
    # Precision or positive predictive value
    PPV = TP/(TP+FP)
    # Negative predictive value
    NPV = TN/(TN+FN)
    # Fall out or false positive rate
    FPR = FP/(FP+TN)
    # False negative rate
    FNR = FN/(TP+FN)
    # False discovery rate
    FDR = FP/(TP+FP)
    # Overall accuracy for each class
    ACC = (TP+TN)/(TP+FP+FN+TN)"""
    
    # Each variable holds the values for every class, so calculate micro avg
    Sensitivity = TP.sum()/ (TP.sum() + FN.sum())
    Specificity = TN.sum()/ (TN.sum() + FP.sum())
    
    print('micro avg -- sensitivity: {}'.format(Sensitivity))
    print('micro avg -- specificity: {}'.format(Specificity))

NOTE: In every confusion matrix: x-axis --> predicted, y-axis --> actual.

test_dl-blur:
Confusion Matrix

[[195  53   2  33   5]
 [  8 176  21  11   7]
 [ 39  33 157  14  15]
 [  1  16  32  59  19]
 [ 18  22  53  21 190]]

Classification Report

              precision    recall  f1-score   support

   Quality 1       0.68      0.75      0.71       261
   Quality 2       0.79      0.59      0.67       300
   Quality 3       0.61      0.59      0.60       265
   Quality 4       0.46      0.43      0.45       138
   Quality 5       0.62      0.81      0.70       236

   micro avg       0.65      0.65      0.65      1200
   macro avg       0.63      0.63      0.63      1200
weighted avg       0.66      0.65      0.64      1200
 samples avg       0.65      0.65      0.65      1200


Summarized Report

Mean precision (without avg values): 0.6328830124823145
Mean recall (without avg values): 0.6317733822567451
Mean f1-score (without avg values): 0.6265583596748644
--------------------------

#### Calculate the error based on absolute difference between y and yhat (Mean Absolute Error)

First of all, the Mean Absolute Error is calculated, which is simply the absolute error between the ground truth and the predicted quality.

Example:

* Ground truth for the quality of three images is $$y = [2, 5, 2]$$
* Model predictions: $$\hat y = [1, 3, 5]$$
* MAE would be: $$MAE(y, \hat y) = \frac{\sum_{i=0}^N |y_{i} - \hat y_{i}|}{N} = \frac{|2 - 1| + |5 - 3| + |2 - 5|}{3} = \frac{6}{3} = 2$$

In [6]:
print("MAE results for each test:\n")
# Loop through the transformed data and calculate MAE
for test_name, results in y_yhats.items():
    print('{}: {:.2f}'.format(test_name, mean_absolute_error(results['prediction'].argmax(axis=1),
                                                             results['ground_truth'].argmax(axis=1))))

MAE results for each test:

test_dl-blur: 0.63
test_ID-blur: 0.78
test_OOD-blur: 1.00
test_dl-ghosting: 0.46
test_ID-ghosting: 0.16
test_OOD-ghosting: 0.12
test_dl-motion: 0.74
test_ID-motion: 0.64
test_OOD-motion: 0.28
test_dl-noise: 0.89
test_ID-noise: 0.29
test_OOD-noise: 0.29
test_dl-resolution: 0.56
test_ID-resolution: 0.28
test_OOD-resolution: 0.22
test_dl-spike: 0.61
test_ID-spike: 0.37
test_OOD-spike: 0.30


In [7]:
print("MAE results for each model:\n")
# Loop through the transformed data and calculate MAE
for (t1, results1), (t2, results2), (t3, results3) in grouper(3, y_yhats.items()):
    model = t1.split('-')[1]
    avg_mae = mean_absolute_error(results1['prediction'].argmax(axis=1),
                                  results1['ground_truth'].argmax(axis=1))
    avg_mae += mean_absolute_error(results2['prediction'].argmax(axis=1),
                                   results2['ground_truth'].argmax(axis=1))
    avg_mae += mean_absolute_error(results3['prediction'].argmax(axis=1),
                                   results3['ground_truth'].argmax(axis=1))
    
    print('Model {}: {:.2f}'.format(model, avg_mae/3))

MAE results for each model:

Model blur: 0.80
Model ghosting: 0.25
Model motion: 0.55
Model noise: 0.49
Model resolution: 0.35
Model spike: 0.43


In a next step, we want an error metric, that shows how far off the model predicts on average. For this, we use the MAE and divide it by the possible number of (quality) classes - 1, here $5 - 1 = 4$. For instance this means that a higher error of two model indicates the predictions of the one model with higher error are less good than the one with the smaller error. So the higher the error, the far off the predictions are from the ground truth values. Maximum error is 100% and would be the case if all images have a quality of 5 and the model predicts a quality of 1 for every image, assuming that the possible values are $(1, 2, 3, 4, 5)$, ie. 5 in total. However the greatest distance a model can achieve is 4 (predict 1 and actual is 5) --> very bad. Let's have a look at a small example:

* Ground truth for the quality of three images is $$y = [2, 5, 2]$$
* Model one makes the following predictions: $$\hat y^{1} = [1, 3, 5]$$
* Model two makes the following predictions: $$\hat y^{2} = [2, 4, 3]$$

Obviously one might correctly assume that model two makes better predictions than model one. Likewise, the error for model two is lower than model one:

* Error for model one: $$\frac{MAE(y, \hat y^{1})}{4} = \frac{2}{4} \Rightarrow 50\%$$ 
* Error for model two: $$\frac{MAE(y, \hat y^{2})}{4} = \frac{0.667}{4} \Rightarrow 16. 67\%$$

In [8]:
print("Error results for each test:\n")

# Loop through the transformed data and calculate the error by dividing by number_classes
for test_name, results in y_yhats.items():
    print('{}: {:.2f}%'.format(test_name, 100*(mean_absolute_error(results['prediction'].argmax(axis=1),
                                                                   results['ground_truth'].argmax(axis=1))/4)))

Error results for each test:

test_dl-blur: 15.71%
test_ID-blur: 19.38%
test_OOD-blur: 24.89%
test_dl-ghosting: 11.58%
test_ID-ghosting: 3.91%
test_OOD-ghosting: 3.12%
test_dl-motion: 18.52%
test_ID-motion: 16.09%
test_OOD-motion: 6.92%
test_dl-noise: 22.19%
test_ID-noise: 7.34%
test_OOD-noise: 7.37%
test_dl-resolution: 14.00%
test_ID-resolution: 7.03%
test_OOD-resolution: 5.47%
test_dl-spike: 15.21%
test_ID-spike: 9.22%
test_OOD-spike: 7.59%


In [9]:
print("Average error results for each model:\n")

# Loop through the transformed data and calculate MAE
for (t1, results1), (t2, results2), (t3, results3) in grouper(3, y_yhats.items()):
    model = t1.split('-')[1]
    error = mean_absolute_error(results1['prediction'].argmax(axis=1),
                                results1['ground_truth'].argmax(axis=1))/4
    error += mean_absolute_error(results2['prediction'].argmax(axis=1),
                                 results2['ground_truth'].argmax(axis=1))/4
    error += mean_absolute_error(results3['prediction'].argmax(axis=1),
                                 results3['ground_truth'].argmax(axis=1))/4
    
    print('Model {}: {:.2f}%'.format(model, 100*error/3))

Average error results for each model:

Model blur: 19.99%
Model ghosting: 6.20%
Model motion: 13.84%
Model noise: 12.30%
Model resolution: 8.83%
Model spike: 10.67%


In [10]:
print("Standard deviation of predictions based on actual labels:\n")

# Loop through the transformed data and calculate the std
for test_name, results in y_yhats.items():
    std = np.std(np.abs(results['prediction'].argmax(axis=1) - results['ground_truth'].argmax(axis=1)))
    print('{}: {:.2f}'.format(test_name, std))

Standard deviation of predictions based on actual labels:

test_dl-blur: 1.00
test_ID-blur: 0.66
test_OOD-blur: 0.77
test_dl-ghosting: 0.94
test_ID-ghosting: 0.66
test_OOD-ghosting: 0.57
test_dl-motion: 0.97
test_ID-motion: 0.63
test_OOD-motion: 0.50
test_dl-noise: 1.04
test_ID-noise: 0.54
test_OOD-noise: 0.51
test_dl-resolution: 1.10
test_ID-resolution: 0.78
test_OOD-resolution: 0.66
test_dl-spike: 0.89
test_ID-spike: 1.10
test_OOD-spike: 0.79
